<a href="https://colab.research.google.com/github/Chiebukar/AGER/blob/main/Age_Gender_webcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

open with colab for better view

Follow link (https://colab.research.google.com/drive/1x9D5hfo07ZsKlSUXsP1D10HYAPlBG17j?authuser=2#scrollTo=e6Ey3U7cAWAj) to use with webcam


go to cell( https://colab.research.google.com/drive/1x9D5hfo07ZsKlSUXsP1D10HYAPlBG17j?authuser=2#scrollTo=ftjdHfXrDKVp&line=6&uniqifier=1) to test with image from webcam


go to cell (https://colab.research.google.com/drive/1x9D5hfo07ZsKlSUXsP1D10HYAPlBG17j?authuser=2#scrollTo=eZbfHSUALaT1&line=6&uniqifier=1) to use video from webcam 

In [ ]:
# import relevant libraries
import numpy as np
import cv2
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.models import load_model
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
from google.colab.patches import cv2_imshow
import PIL
import io
import html
import time
import os
import matplotlib.pyplot as plt
%matplotlib inline

# Setting up the models

In [ ]:
# get path to models
path = '/content/drive/MyDrive/Colab Notebooks/models/'
ageModelPath = os.path.join(path, 'age_model.h5')
genderModelPath = os.path.join(path, 'gender_model.h5')

In [ ]:
# get models
age_model = load_model(ageModelPath)
gender_model = load_model(genderModelPath)

In [ ]:
# define class dictionries
gender_dict = {0: 'Female', 1: 'Male'}
 
# face detector
face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/haarcascade_frontalface_default.xml')

In [ ]:
# get label icons
icons_folder = '/content/drive/MyDrive/Colab Notebooks/icons/'
icon_dict = {}
 
for icon in os.listdir(icons_folder):
  icon_path = os.path.join(icons_folder+icon)
  icon_name = icon.split('.')[0]
  icon_img = cv2.imread(icon_path)
  icon_dict[icon_name] = icon_img

In [ ]:
icon_dict.keys()

# Image Data Preprocessing

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Converts Javascript image object to Opencv image
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)
 
  return img

In [ ]:
# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
 
  return bbox_bytes

In [ ]:
# fuction to preprocess image for classification
def preprocess_image(img):
    img = cv2.resize(img, (224,224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255
    return img

# Image Classification

In [ ]:
# function to run classifiers on image
def classify_image(img):
  img_pixels = preprocess_image(img)
 
  # predict age
  output_indices = np.array(range(101))     # indices for age labels
  age_prediction = age_model.predict(img_pixels)
  age = str(int(np.floor(np.sum(age_prediction * output_indices, axis = 1))[0]))
 
  # gender prediction
  gender_prediction = gender_model.predict(img_pixels)
  gender_index = np.argmax(gender_prediction)
  gender = gender_dict[gender_index]
 
  return age, gender

In [ ]:
def putText(age,gender,bbox_array,x,y,w,h, video = False):
  boxColourTop = (46,200,255)
  boxColourSide = (0,0,0)
 
  overlay = bbox_array.copy()
  alpha = 0.0
 
  gender_icon = icon_dict[gender]
  
 
  if video:
    gender_icon = cv2.cvtColor(gender_icon, cv2.COLOR_RGB2RGBA)
 
 
  boxTriangle = np.array([(x+int(w/2), y), (x+int(w/2)-20, y-20), (x+int(w/2)+20, y-20)])
  cv2.drawContours(bbox_array, [boxTriangle], 0, boxColourTop, -1)
  bbox_array = cv2.rectangle(bbox_array,(x+int(w/2)-50,y-20),(x+int(w/2)+50,y-90),boxColourTop,cv2.FILLED)
 
  bbox_array = cv2.putText(bbox_array, "{}".format(age), 
                  (x+int(w/2), y - 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
 
  bbox_array[y-75:y-75+gender_icon.shape[0], x+int(w/2)-45:x+int(w/2)-45+gender_icon.shape[1]] = gender_icon

# Setting up webcam for capturing Images

In [ ]:
def take_photo(quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);
 
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
 
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
 
      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
 
      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);
 
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
 
  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data) 
  # grayscale img
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  # get face bounding box coordinates using Haar Cascade
  faces = face_cascade.detectMultiScale(gray)
  return img, faces

In [ ]:
def image_output(filename='photo.jpg'):
 
  img, faces = take_photo(quality=0.8)
  for (x,y,w,h) in faces:
      img = cv2.rectangle(img,(x,y),(x+w,y+h),(220,220,220),1)
      detected_face = img[int(y): int(y+h), int(x): int(x+w)]
      
      # get predicted labels
      age, gender = classify_image(detected_face)
 
      # 
      putText(age,gender,img,x,y,w,h, video)
 
  # save image
  cv2.imwrite(filename, img)
  return filename

# Setting up webcam for video stream

In [ ]:
# JavaScript to  create live video stream using webcam 
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
 
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
 
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    async function createDom() {
      if (div !== null) {
        return stream;
      }
 
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);
 
      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();
 
      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }
      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
 
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')
  display(js)

In [ ]:
# function to get javascript image data from web cam
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# Test webcam for images

In [ ]:
# uncomment cell to use

# video = False
# try:
#   filename = image_output('photo.jpg')
#   print('Saved to {}'.format(filename))
  
#   # Show the image which was just taken.
#   display(Image(filename))
# except Exception as err:
#   # Errors will be thrown if the user does not have a webcam or if they do not
#   # grant the page permission to access it.
#   print('error: ',str(err))

# Test webcam for video stream






In [ ]:
#uncomment cell to use

# video= True
# # start streaming video from webcam
# video_stream()
# # label for video
# label_html = 'Capturing...'
# # initialze bounding box to empty
# bbox = ''
# count = 0 
# while True:
#   js_reply = video_frame(label_html, bbox)
#   if not js_reply:
#     break
 
#   # convert JS response to OpenCV Image
#   img = js_to_image(js_reply["img"])
 
#   # create transparent overlay for bounding box
#   bbox_array = np.zeros([480,640,4], dtype=np.uint8)
#   # grayscale image for face detection
#   gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
 
#   # get face region coordinates
#   faces = face_cascade.detectMultiScale(gray)
#   # get face bounding box for overlay
#   for (x,y,w,h) in faces:
#     bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)
#     detected_face = img[int(y): int(y+h), int(x): int(x+w)] # crop detected face
 
#     try:
#       age, gender = classify_image(detected_face) # classify image and get labels
      
 
#       # add labels to image
#       putText(age,gender,bbox_array,x,y,w,h,video)
    
      
#     except Exception as err:
#       print(str(err))      # show errors if encountered 
        
#   bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int)  * 255
#   # convert overlay of bbox into bytes
#   bbox_bytes = bbox_to_bytes(bbox_array)
#   # update bbox so next frame gets new overlay
#   bbox = bbox_bytes